In [12]:
from pydantic import BaseModel, Field
from typing import Optional

class UserRegistration(BaseModel):
    # '...' means the field is REQUIRED
    username: str = Field(..., min_length=5, max_length=15, pattern="^[a-zA-Z0-9]+$")
    age: int = Field(..., ge=18, le=100, description="Must be an adult")
    is_active: bool = Field(default=True)


try:
    
    username = str(input("enter the username"))
    age = int(input("enter your age"))
    user=UserRegistration(username=username, age=age)
    print(f"✅ User  created: {user.model_dump()}")

except Exception as e:
    print(f"\n❌ Validation Error:\n{e}")

✅ User  created: {'username': 'dhdudt', 'age': 33, 'is_active': True}


In [2]:
from pydantic import BaseModel, field_validator

class JobApplication(BaseModel):
    role: str
    salary_ask: float

    # Check the Role
    @field_validator('role')
    @classmethod
    def check_role(cls, v: str) -> str:
        allowed = ["Data Scientist", "ML Engineer"]
        if v not in allowed:
            raise ValueError(f"Sorry, we are only hiring for: {allowed}")
        return v

    # Check the Salary
    @field_validator('salary_ask')
    @classmethod
    def check_budget(cls, v: float) -> float:
        if v > 200000:
            raise ValueError("Salary expectation exceeds our budget.")
        return v

# --- TEST IT ---
try:
    # 1. Valid Application
    app1 = JobApplication(role="Data Scientist", salary_ask=150000)
    print(f"✅ Application accepted for: {app1.role}")

    # 2. Invalid Role
    app2 = JobApplication(role="Web Developer", salary_ask=100000)
except Exception as e:
    print(f"\n❌ Application Rejected:\n{e}")

✅ Application accepted for: Data Scientist

❌ Application Rejected:
1 validation error for JobApplication
role
  Value error, Sorry, we are only hiring for: ['Data Scientist', 'ML Engineer'] [type=value_error, input_value='Web Developer', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error


In [ ]:
from pydantic import BaseModel, model_validator
from datetime import date

class HotelBooking(BaseModel):
    check_in: date
    check_out: date

    @model_validator(mode='after')
    def check_dates(self):
        # In V2, 'self' allows us to access fields directly
        if self.check_out <= self.check_in:
            raise ValueError("Check-out date must be AFTER check-in date")
        return self

# --- TEST IT ---
try:
    # 1. Valid Booking
    booking1 = HotelBooking(check_in=date(2026, 5, 1), check_out=date(2026, 5, 20))
    print(f"✅ Booking Confirmed: {booking1.check_in} to {booking1.check_out}")

    # 2. Invalid Booking 
    # Check out is BEFORE Check in
    booking2 = HotelBooking(check_in=date(2026, 5, 20), check_out=date(2026, 5, 1))
except Exception as e:
    print(f"\n❌ Booking Error:\n{e}")

✅ Booking Confirmed: 2026-05-01 to 2026-05-20

❌ Booking Error:
1 validation error for HotelBooking
  Value error, Check-out date must be AFTER check-in date [type=value_error, input_value={'check_in': datetime.dat...tetime.date(2026, 5, 1)}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error
